## Retriever And Chain With Langchain


In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()

# os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")

In [2]:
# from langchain_community.document_loaders import PyPDFLoader

# loader = PyPDFLoader("Jenkins-Readme.pdf")
# docs = loader.load()
# docs

[Document(page_content='SimplifyQA Pipeline Connector is an update to our existing \nJenkins Plugin with improved features.  It offers seamless \nintegration with SimplifyQA tool and triggers automated \nscripts execution.  \nThis plugin must be used in combination with SimplifyQA automation tool. It supports from Jenkins \nversion –  \nNew features that are added to this improved utility are controlling the Build Failure status based on \nthe percentage of failed testcases, improved timeouts and support to different types of Jenkins \nproject.  \nFollow below steps for Integration  \n1. SimplifyQA Agent should be installed and registered in the Host system. Login to \nSimplifyQA application, follow the steps in Help Guide to Download and install \nthe Agent and register the machine in which installation is done.  \n2. Create Suite of automated testcases that needs to be executed as part of CICD \npipeline.  \n3. Create a Pipeline with required suite and provide details of registered m

In [ ]:
from git import Repo
from langchain_community.document_loaders.generic import GenericLoader
from langchain_community.document_loaders.parsers import LanguageParser
from langchain_text_splitters import Language


In [ ]:
repo_path

if os.path.isdir(repo_path):  # if repo exists, pull newest data
    repo = Repo(repo_path)
    repo.remotes.origin.pull()
else:  # otherwise, clone from remote
    repo = Repo.clone_from("https://github.com/jayggg/mth271content", repo_path)

In [ ]:
# Load
loader = GenericLoader.from_filesystem(
    repo_path + "/libs/core/langchain_core",
    glob="**/*",
    suffixes=[".py"],
    exclude=["**/non-utf8-encoding.py"],
    parser=LanguageParser(language=Language.JAVA, parser_threshold=500),
)
documents = loader.load()
len(documents)

In [3]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
text_splitter.split_documents(docs)[:5]

[Document(page_content='SimplifyQA Pipeline Connector is an update to our existing \nJenkins Plugin with improved features.  It offers seamless \nintegration with SimplifyQA tool and triggers automated \nscripts execution.  \nThis plugin must be used in combination with SimplifyQA automation tool. It supports from Jenkins \nversion –  \nNew features that are added to this improved utility are controlling the Build Failure status based on \nthe percentage of failed testcases, improved timeouts and support to different types of Jenkins \nproject.  \nFollow below steps for Integration  \n1. SimplifyQA Agent should be installed and registered in the Host system. Login to \nSimplifyQA application, follow the steps in Help Guide to Download and install \nthe Agent and register the machine in which installation is done.  \n2. Create Suite of automated testcases that needs to be executed as part of CICD \npipeline.  \n3. Create a Pipeline with required suite and provide details of registered m

In [4]:
documents = text_splitter.split_documents(docs)
documents

[Document(page_content='SimplifyQA Pipeline Connector is an update to our existing \nJenkins Plugin with improved features.  It offers seamless \nintegration with SimplifyQA tool and triggers automated \nscripts execution.  \nThis plugin must be used in combination with SimplifyQA automation tool. It supports from Jenkins \nversion –  \nNew features that are added to this improved utility are controlling the Build Failure status based on \nthe percentage of failed testcases, improved timeouts and support to different types of Jenkins \nproject.  \nFollow below steps for Integration  \n1. SimplifyQA Agent should be installed and registered in the Host system. Login to \nSimplifyQA application, follow the steps in Help Guide to Download and install \nthe Agent and register the machine in which installation is done.  \n2. Create Suite of automated testcases that needs to be executed as part of CICD \npipeline.  \n3. Create a Pipeline with required suite and provide details of registered m

In [5]:
# from langchain_community.embeddings import OpenAIEmbeddings
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS

# db = FAISS.from_documents(documents[:30], OpenAIEmbeddings())
db = FAISS.from_documents(documents[:30], OllamaEmbeddings())

In [6]:
db

In [8]:
query = "What is Simplifyqa pipeline connector?"
result = db.similarity_search(query)
result[0].page_content

'10. Results would also be available in SimplifyQA reports section. There would also \nbe an easy navigation to SimplifyQA reports section from logs. Users can view the \nreport and download as PDF.  \n11. Tool can also trigger email to mentioned users, post completion of execution.'

In [9]:
from langchain_community.llms import Ollama

## Load Ollama LAMA2 LLM model
llm = Ollama(model="llama2")
llm

Ollama()

In [10]:
## Design ChatPrompt Template
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template(
    """
Answer the following question based only on the provided context. 
Think step by step before providing a detailed answer. 
I will tip you $1000 if the user finds the answer helpful. 
<context>
{context}
</context>
Question: {input}"""
)

In [11]:
## Chain Introduction
## Create Stuff Docment Chain

from langchain.chains.combine_documents import create_stuff_documents_chain

document_chain = create_stuff_documents_chain(llm, prompt)

In [12]:
"""
Retrievers: A retriever is an interface that returns documents given
 an unstructured query. It is more general than a vector store.
 A retriever does not need to be able to store documents, only to 
 return (or retrieve) them. Vector stores can be used as the backbone
 of a retriever, but there are other types of retrievers as well. 
 https://python.langchain.com/docs/modules/data_connection/retrievers/   
"""

retriever = db.as_retriever()
retriever

VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000013CF7040550>)

In [13]:
"""
Retrieval chain:This chain takes in a user inquiry, which is then
passed to the retriever to fetch relevant documents. Those documents 
(and original inputs) are then passed to an LLM to generate a response
https://python.langchain.com/docs/modules/chains/
"""

from langchain.chains import create_retrieval_chain

retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [16]:
response = retrieval_chain.invoke({"input": "SimplifyQA pipeline connector"})

In [17]:
response["answer"]

'Based on the provided context, the SimplifyQA pipeline connector is an update to the existing Jenkins Plugin that offers seamless integration with SimplifyQA tool and triggers automated scripts execution. The plugin must be used in combination with SimplifyQA automation tool and supports various features such as:\n\n1. Controlling build failure status based on the percentage of failed testcases.\n2. Improved timeouts.\n3. Support for different types of Jenkins projects.\n\nTo integrate SimplifyQA pipeline connector with Jenkins, follow these steps:\n\n1. Install and register SimplifyQA Agent in the host system.\n2. Create a suite of automated testcases that need to be executed as part of the CICD pipeline.\n3. Create a pipeline with the required suite and provide details of the registered machine.\n4. Enter failure percentage and enable verbose flag as required.\n5. Save the details.\n6. Post-build, Jenkins will trigger the execution of the mentioned suite in the registered machine.\n